```Packages and given datasets```

In [84]:
import pandas as pd
import json
from datetime import datetime

***Restaurant Dataset***

In [85]:
# Open file and convert to dataframe (df)
with open('restaurant_data.json', encoding="utf8") as rest_file:
    rest_data = json.load(rest_file)

rest_df = pd.DataFrame(rest_data)

print(rest_df.info())
# Check first few rows of df
print(rest_df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   results_found  79 non-null     int64 
 1   results_start  79 non-null     int64 
 2   results_shown  79 non-null     int64 
 3   restaurants    79 non-null     object
dtypes: int64(3), object(1)
memory usage: 2.6+ KB
None
   results_found  results_start  results_shown  \
0          29287              1             20   
1           7625              1             20   
2          21776              1             20   
3          16762              1             20   
4          12026              1             20   

                                         restaurants  
0  [{'restaurant': {'R': {'res_id': 18649486}, 'a...  
1  [{'restaurant': {'R': {'res_id': 18707652}, 'a...  
2  [{'restaurant': {'R': {'res_id': 18392725}, 'a...  
3  [{'restaurant': {'R': {'res_id': 58882}, 'apik...  
4  [{'res

***Country-Code Dataset***

In [86]:
country_df = pd.read_excel("Country-code.xlsx")
print(country_df.info())
print(country_df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Code  15 non-null     int64 
 1   Country       15 non-null     object
dtypes: int64(1), object(1)
memory usage: 372.0+ bytes
None
   Country Code       Country
0             1         India
1            14     Australia
2            30        Brazil
3            37        Canada
4            94     Indonesia
5           148   New Zealand
6           162   Phillipines
7           166         Qatar
8           184     Singapore
9           189  South Africa


***Checking all features of a restaurant***

In [87]:
# Function to retrieve all keys from each restaurant
def get_all_keys(dic):
    keys = set(dic.keys())
    for value in dic.values():
        if isinstance(value, dict):
            keys.update(get_all_keys(value))
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    keys.update(get_all_keys(item))
    return keys

common_keys = set()
add_keys = set()

# Check which keys are common for each restaurant and which are additional keys
for rest_list in rest_df["restaurants"]:
    for rest in rest_list:
        rest_keys = set(get_all_keys(rest))

        if not common_keys:
            common_keys = rest_keys
        else:
            common_keys = common_keys.intersection(rest_keys)

        add_keys = add_keys.union(rest_keys)

add_keys -= common_keys

print("Common Keys:")
print(common_keys)
print("\nAdditional Keys:")
print(add_keys)

Common Keys:
{'city_id', 'longitude', 'events_url', 'location', 'zipcode', 'country_id', 'include_bogo_offers', 'name', 'res_id', 'locality', 'deeplink', 'offers', 'average_cost_for_two', 'user_rating', 'has_table_booking', 'is_table_reservation_supported', 'photos_url', 'url', 'menu_url', 'switch_to_order_menu', 'apikey', 'city', 'cuisines', 'price_range', 'currency', 'establishment_types', 'votes', 'has_fake_reviews', 'restaurant', 'is_delivering_now', 'rating_color', 'address', 'latitude', 'thumb', 'featured_image', 'aggregate_rating', 'R', 'locality_verbose', 'rating_text', 'has_online_delivery', 'id'}

Additional Keys:
{'mezzo_provider', 'share_url', 'voucher_id', 'should_show', 'start_date', 'display_date', 'rating_tool_tip', 'zomato_events', 'impressions', 'type', 'is_zomato_book_res', 'is_zomato_event', 'type_code', 'book_link', 'restaurants', 'friendly_end_date', 'is_valid', 'description', 'restaurant_list', 'offer', 'book_again_url', 'is_end_time_set', 'display_time', 'uuid',

1. Extract the following fields and store the data as restaurants.csv. <br>
◦   	Restaurant Id<br>
◦   	Restaurant Name<br>
◦   	Country<br>
◦   	City<br>
◦   	User Rating Votes<br>
◦   	User Aggregate Rating (in float)<br>
◦   	Cuisines<br>


In [88]:
print(f"Number of Restaurants per row: {len((rest_df.iloc[0]['restaurants']))}")
print(f"Restaurant example: {list(rest_df['restaurants'][1][0]['restaurant'].keys())}")

result = []

for rest_list in rest_df["restaurants"]:
    for rest in rest_list:
        curr = rest['restaurant']
        rest_id = curr['id']
        name = curr["name"]
        country_id =  curr["location"]["country_id"]
        city = curr["location"]["city"]
        rating_votes = curr["user_rating"]["votes"]
        agg_user_rating = curr["user_rating"]["aggregate_rating"]
        cuisines = curr["cuisines"]
        temp = [rest_id, name, country_id, city, rating_votes, agg_user_rating, cuisines]
        result.append(temp)

result = pd.DataFrame(result, columns= ['Restaurant Id', "Restaurant Name", "Country_id", 
                                        "City", "User Rating Votes",
                                        "User Aggregate Rating", "Cuisines"])

   

Number of Restaurants per row: 20
Restaurant example: ['R', 'apikey', 'id', 'name', 'url', 'location', 'switch_to_order_menu', 'cuisines', 'average_cost_for_two', 'price_range', 'currency', 'offers', 'opentable_support', 'is_zomato_book_res', 'mezzo_provider', 'is_book_form_web_view', 'book_form_web_view_url', 'book_again_url', 'thumb', 'user_rating', 'photos_url', 'menu_url', 'featured_image', 'has_online_delivery', 'is_delivering_now', 'has_fake_reviews', 'include_bogo_offers', 'deeplink', 'is_table_reservation_supported', 'has_table_booking', 'events_url', 'establishment_types']


In [89]:
result_df = pd.merge(result, country_df, left_on="Country_id", right_on="Country Code")
result_df = result_df.drop(columns=["Country_id", "Country Code"])
result_df = result_df[['Restaurant Id', "Restaurant Name", "Country", 
                                        "City", "User Rating Votes",
                                        "User Aggregate Rating", "Cuisines"]]

result_df.to_csv("restaurants.csv", index=False)   

2. 	Extract the list of restaurants that have past event in the month of April 2019 and store the data as restaurant_events.csv. <br>
◦   	Event Id <br>
◦   	Restaurant Id <br>
◦   	Restaurant Name <br>
◦   	Photo URL <br>
◦   	Event Title <br>
◦   	Event Start Date <br>
◦   	Event End Date <br>
Note: Populate empty values with "NA". <br>


In [90]:
result_2 = []

fixed_date = datetime(2019, 4, 1)

for rest_list in rest_df["restaurants"]:
    for rest in rest_list:
        curr = rest['restaurant']
        rest_id = curr['id']
        name = curr["name"]
        photo_url = curr["photos_url"]

        events = curr.get("zomato_events", False)
        if events:
            for event in events:
                event_det = event["event"]
                event_id = event_det["event_id"]
                event_title = event_det["title"]
                event_start_date = event_det["start_date"]
                event_end_date = event_det["end_date"]

                event_start_date_dt = datetime.strptime(event_start_date, "%Y-%m-%d")
                event_end_date_dt = datetime.strptime(event_end_date, "%Y-%m-%d")
                
                if event_start_date_dt <= fixed_date <= event_end_date_dt:
                    temp = [event_id, rest_id, name, photo_url, event_title, event_start_date, event_end_date]
                    result_2.append(temp)

rest_event_df = pd.DataFrame(result_2, columns= ["Event_id", 'Restaurant Id', "Restaurant Name", "Photo_URL", 
                                        "Event Title", "Event Start Date",
                                        "Event_End_Date"])

rest_event_df.to_csv("restaurant_events.csv", index = False)

3. 	From the dataset (restaurant_data.json), determine the threshold for the different rating text based on aggregate rating. Return aggregates for the following ratings only: <br>
◦   	Excellent <br>
◦   	Very Good <br>
◦   	Good <br>
◦   	Average <br>
◦   	Poor <br>


In [92]:
rating = {}
selected_rate = ["Excellent", "Very Good", "Good", 'Average', "Poor"]

for rest_list in rest_df["restaurants"]:
    for rest in rest_list:
        user_rating = rest['restaurant']["user_rating"]
        aggre_rating = float(user_rating['aggregate_rating'])
        rate_text = user_rating["rating_text"]

        if rate_text in rating:
            rating[rate_text] = [min(aggre_rating, rating[rate_text][0]), max(aggre_rating, rating[rate_text][1])]
        else:
            rating[rate_text] = [aggre_rating, aggre_rating]

rating = list(rating.items())

filtered_rating = list(filter(lambda x: x[0] in selected_rate, rating))
sorted_rating = sorted(filtered_rating, key = lambda x: x[1][1], reverse=True)

for word, spread in sorted_rating:
    print(f"The '{word}' rating has a minimum and maximum rating threshold of {spread} respectively")


The 'Excellent' rating has a minimum and maximum rating threshold of [4.5, 4.9] respectively
The 'Very Good' rating has a minimum and maximum rating threshold of [4.0, 4.4] respectively
The 'Good' rating has a minimum and maximum rating threshold of [3.5, 3.9] respectively
The 'Average' rating has a minimum and maximum rating threshold of [2.5, 3.4] respectively
The 'Poor' rating has a minimum and maximum rating threshold of [2.2, 2.2] respectively


Based on the given data, <br>
The 'Excellent' rating has a minimum and maximum rating threshold of [4.5, 4.9] respectively. <br>
The 'Very Good' rating has a minimum and maximum rating threshold of [4.0, 4.4] respectively. <br>
The 'Good' rating has a minimum and maximum rating threshold of [3.5, 3.9] respectively. <br>
The 'Average' rating has a minimum and maximum rating threshold of [2.5, 3.4] respectively. <br>
The 'Poor' rating has a minimum and maximum rating threshold of [2.2, 2.2] respectively. <br>

However, based on the rating threshold patterns, we can assume that the "Poor" rating has a maximum threshold of 2.4